Вариант 2. Выполнение заявок. Автомат выполняет заявки из N очередей (N произвольное, должно задаваться). Каждый такт в одну из очередей поступает заявка. Автомат выбирает одну из очередей и выполняет заявку из неё. Если в очереди есть заявка, автомат получает награду. Если заявки нет, автомат получает наказание.

Реализуйте сценарий для фиксированной вероятности распределения заявок по очередям, (случайно генерируется и направление, и количество). Определите, какие стратегии (осторожная/доверчивая/равномерная, инертная или не инертна) наиболее правильны для разных случаев. Для минимизации ожидания заявок или максимизации награды

In [9]:
import random

class State:  # состояние 
    # ['0.5>+>1', '0.5>0']  
    # до стрелки ('>') укажем вероятность перехода
    # после стрелки укажем направление перехода
    def __init__(self, state_id, edges, out):
        self.state_id = state_id
        self.edges = edges
        self.out = out

    def __str__(self):
        return str(self.state_id) + ' : ' + str(self.edges)

    def next(self):
        rnd = random.random()  # генерирует число в диапазоне [0; 1)
        for edge in self.edges:
            if rnd < float(edge.split('>')[0]):
                return edge.split('>')[2], edge.split('>')[1]
            else:
                rnd -= float(edge.split('>')[0])

class PA:  # ВА - вероятностный (конечный) автомат
    
    def __init__(self, name, states):
        self.name = name
        self.states = states
        self.current_state = self.states[0]
    
    def reset(self):
        self.current_state = self.states[0]
    
    def __str__(self):
        return '#PA "' + self.name + '" ' + str(self.current_state)
    
    def compute(self, n):  # n - количество шагов
        word = ''
        for _ in range(n):
            next_id, action = self.current_state.next()  # следующее состояние
            for state in self.states:
                if state.state_id == next_id:
                    self.current_state = state
            word += action + ' : ' + self.current_state.state_id + ' ' + self.current_state.out + '\n'
        return word
    
    def change_states(self, new_states):
        self.states = new_states
        for state in self.states:
            if state.state_id == self.current_state.state_id:
                self.current_state = state

def build_states(ps):
    states = []
    D = 10  # инерция - длина лепестков (насколько быстро выйдем из неправильной очереди)
    punish_step = 1
    reward_step = 1
    for queue_ind in range(len(ps)):
        for i in range(D):
            edges = []
            if i > 0:
                edges.append(f'{ps[queue_ind]}>+>Q{queue_ind}{min(i + reward_step, D - 1)}')
                edges.append(f'{1 - ps[queue_ind]}>->Q{queue_ind}{max(0, i - punish_step)}')
            else:
                edges.append(f'{ps[queue_ind]}>+>Q{queue_ind}{reward_step}')
                for queue_other in range(len(ps)):
                    if queue_other != queue_ind:
                        edges.append(f'{(1 - ps[queue_ind]) / (len(ps) - 1)}>->Q{queue_other}0')
            state = State(f'Q{queue_ind}{i}', edges, f'Q{queue_ind}')
            states.append(state)
    return states

# Задание вероятностей и создание состояний
# ps = [0.1, 0.1, 0.2, 0.2, 0.3, 0.3, 0.4, 0.4, 0.5, 0.5]
# states = build_states(ps)
# pa = PA('Auto', states)
# steps = 10
# log = pa.compute(steps)
# print(log)
# win = log.count('+')
# print(win, ' : ', win / steps)

def hand_control():
# #     Устанавливаем фиксированное количество очередей
#     num = random.randint(2, 9)
#     values = [0.1, 0.1, 0.1, 0.2, 0.2, 0.05, 0.05, 0.1, 0.1]
#     gen = values[:num]
#     total = sum(gen)
#     gen = [value / total for value in gen]
    

   
    num = 5
    # Устанавливаем вероятности переходов между очередями
    gen = [0.1, 0.3, 0.4, 0.05, 0.15]
    
    # Создаем экземпляр класса PA (Probabilistic Automaton) с названием 'Automat'
    # и передаем ему состояние с вероятностями
    automat = PA('Automat', build_states(gen))

    # Бесконечный цикл для ожидания ввода пользователем команды
    while True:
        # Ждем ввод команды от пользователя
        command = input("Введите команду (STOP для выхода): ").upper()

        # Проверяем, если пользователь вводит "STOP", прекращаем цикл
        if command == "STOP":
            break

        try:
            # Пытаемся извлечь индекс очереди, уменьшая его на 1,
            # чтобы соответствовать нумерации индексов списка (начиная с 0)
            queue_index = int(command[0]) - 1
            # Остальные символы команды преобразуем в количество шагов автомата
            steps = int(command[1:])
            
            # Создаем список вероятностей, все значения которого по умолчанию 0
            ps = [0] * num
            # Устанавливаем вероятность выбранной очереди в 1
            ps[queue_index] = 1
            # Изменяем состояния автомата под новую вероятность (только выбранная очередь имеет вероятность перейти)
            automat.change_states(build_states(ps))

            # Выполняем вычисления автоматом на указанное количество шагов
            log = automat.compute(steps)
            # Выводим результат вычислений (лог) в консоль
            print(log)
        
        except (IndexError, ValueError):
            # Обрабатываем возможные ошибки: неверный индекс или некорректное значение
            print("Некорректная команда. Попробуйте снова.")

# Запускает функцию hand_control() для интерактивной работы с пользователем
hand_control()

Введите команду (STOP для выхода): 123
+ : Q01 Q0
+ : Q02 Q0
+ : Q03 Q0
+ : Q04 Q0
+ : Q05 Q0
+ : Q06 Q0
+ : Q07 Q0
+ : Q08 Q0
+ : Q09 Q0
+ : Q09 Q0
+ : Q09 Q0
+ : Q09 Q0
+ : Q09 Q0
+ : Q09 Q0
+ : Q09 Q0
+ : Q09 Q0
+ : Q09 Q0
+ : Q09 Q0
+ : Q09 Q0
+ : Q09 Q0
+ : Q09 Q0
+ : Q09 Q0
+ : Q09 Q0

Введите команду (STOP для выхода): 421
- : Q08 Q0
- : Q07 Q0
- : Q06 Q0
- : Q05 Q0
- : Q04 Q0
- : Q03 Q0
- : Q02 Q0
- : Q01 Q0
- : Q00 Q0
- : Q30 Q3
+ : Q31 Q3
+ : Q32 Q3
+ : Q33 Q3
+ : Q34 Q3
+ : Q35 Q3
+ : Q36 Q3
+ : Q37 Q3
+ : Q38 Q3
+ : Q39 Q3
+ : Q39 Q3
+ : Q39 Q3

Введите команду (STOP для выхода): STOP
